In [9]:
import numpy as np
import pandas as pd
import string
import re
import random
import pickle

In [10]:
with open("businessCard.txt", mode = "r", encoding = "utf8", errors = "ignore") as f:
    text = f.read()
dataList = list(map(lambda x:x.split("\t"),text.split("\n")))
dataFrame = pd.DataFrame(dataList[1:], columns = dataList[0]) 
dataFrame.dropna(inplace=True)

In [11]:
whiteSpace = string.whitespace
punctuation = '!#$%&\'()*+:;<=>?[\\]^`{|}~'
tableWhiteSpace = str.maketrans('','', whiteSpace)
tablePunctuation = str.maketrans('','', punctuation)
def cleanText(text):
    text = str(text)
    text = text.lower()
    removeWhiteSpace = text.translate(tableWhiteSpace)
    removePunctuation = removeWhiteSpace.translate(tablePunctuation)

    return str(removePunctuation)

In [13]:
dataFrame["text"] = dataFrame["text"].apply(cleanText)
dataClean = dataFrame.query("text != ''")
dataClean.dropna(inplace = True)

C:\OCR-Business_card\cardReader\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [14]:
group = dataClean.groupby(by = "id")
groupArray = group.get_group('000.jpeg')[["text","tag"]].values
content = ""
annotations = {"entities": []}
start = 0
end = 0

for text, label in groupArray:
    text = str(text)
    stringLen = len(text) + 1

    start = end
    end = start + stringLen

    if label != "O":
        annot = (start, end-1,label)
        annotations["entities"].append(annot)

    content = content + text + " "

In [15]:
allCardsData = []
group = dataClean.groupby(by = "id")
cards = group.groups.keys()
for card in cards:
    cardData = []
    groupArray = group.get_group(card)[["text","tag"]].values
    content = ""
    annotations = {"entities": []}
    start = 0
    end = 0

    for text, label in groupArray:
        text = str(text)
        stringLen = len(text) + 1

        start = end
        end = start + stringLen

        if label != "O":
            annot = (start, end-1,label)
            annotations["entities"].append(annot)

        content = content + text + " "

    cardData = (content, annotations)
    allCardsData.append(cardData)

In [16]:
random.shuffle(allCardsData)
TrainData = allCardsData[:240]
TestData = allCardsData[240:]

In [17]:
pickle.dump(TrainData, open("./data/TrainData.pickle", mode = "wb"))
pickle.dump(TestData, open("./data/TestData.pickle", mode = "wb"))